In [1]:
import pickle
import pandas as pd
import numpy as np
import urllib, time, json, ssl

In [2]:
pd.options.display.max_columns = 120

In [3]:
df = pd.read_pickle('Data_Step1_8')

FileNotFoundError: [Errno 2] No such file or directory: 'Data_Step1_8'

In [5]:
df_uncod = df.loc[(df.lat == 0) | (df.lng == 0), ['DataId','State', 'County','City', 'Address','PostalCode','Location','lat','lng']]

In [6]:
df_uncod.sort_values('Location', na_position= 'first')

,DataId,State,County,City,Address,PostalCode,Location,lat,lng
10942,648228,CA,Ventura,Westlake Village,1188 GALESMORE Court,91361,zz_ZERO_RESULTS,0.0,0.0
11883,1796667,CA,Madera,Madera,1611 Jennings (3rd) Street,93637,zz_ZERO_RESULTS,0.0,0.0
12092,652598,CA,Riverside,Lake Mathews,21274 Ridgedale Drive,92570,zz_ZERO_RESULTS,0.0,0.0
12397,1631098,CA,San Bernardino,Blue Jay,363 Cedar / Blue Jay Canyon Court,92317,zz_ZERO_RESULTS,0.0,0.0
12665,1840311,CA,Riverside,Lake Mathews,17465 Vanderhill Circle,92570,zz_ZERO_RESULTS,0.0,0.0
14442,1795450,CA,Butte,Chico,718 Burnt Ranch (Lot 419) Way,95973,zz_ZERO_RESULTS,0.0,0.0
17725,1895655,CA,Los Angeles,Sunland,10574 SUNLAND BLVD #5 Boulevard,91040,zz_ZERO_RESULTS,0.0,0.0
19079,1879773,CA,Orange,Seal Beach,13140 Del Monte M16-52G Drive,90740,zz_ZERO_RESULTS,0.0,0.0
21849,1980268,CA,San Bernardino,Rancho Cucamonga,8074 Tulsa Place,91730,zz_ZERO_RESULTS,0.0,0.0


In [7]:
len(df_uncod)

9

In [8]:
# Geocoding:
serviceurl = 'https://maps.googleapis.com/maps/api/geocode/json?'
# Deal with SSL certificate anomalies Python > 2.7
scontext = ssl.SSLContext(ssl.PROTOCOL_TLSv1)

In [9]:
df_uncod.Location.value_counts()

zz_ZERO_RESULTS    9
Name: Location, dtype: int64

In [10]:
limit = 2500 if len(df_uncod) > 2500 else len(df_uncod)
city_exceptions = ['Outside Area (Inside Ca)', 'Outside Area (Outside Ca)', None]
county_exceptions = ['Other County', 'Other State']
state_exceptions = ['OR']
# limit = 0  # For testing
count = 0

In [11]:
idnr = df_uncod.iloc[count, df_uncod.columns.get_loc('DataId')]
address = df_uncod.iloc[count, df_uncod.columns.get_loc('Address')]
city = df_uncod.iloc[count, df_uncod.columns.get_loc('City')]
state = df_uncod.iloc[count, df_uncod.columns.get_loc('State')]
zipcode = int(df_uncod.iloc[count, df_uncod.columns.get_loc('PostalCode')])

query = ', '.join((address, city, state))

In [12]:
query

'1188 GALESMORE Court, Westlake Village, CA'

In [13]:
API_key = ''

In [14]:
for count in range(0, len(df_uncod)):
    idnr = df_uncod.iloc[count, df_uncod.columns.get_loc('DataId')]
    address = df_uncod.iloc[count, df_uncod.columns.get_loc('Address')]
    city = df_uncod.iloc[count, df_uncod.columns.get_loc('City')]
    state = df_uncod.iloc[count, df_uncod.columns.get_loc('State')]
    zipcode = int(df_uncod.iloc[count, df_uncod.columns.get_loc('PostalCode')])

    url = serviceurl + urllib.parse.urlencode({"address": query, "key": API_key})

    query = ', '.join((address, city, state))
    uh = urllib.request.urlopen(url, context=scontext)
    entry = uh.read()​uh = urllib.request.urlopen(url, context=scontext)
    js=json.loads(entry)
    print (js['status'])
    time.sleep(0.1)

ZERO_RESULTS
ZERO_RESULTS
ZERO_RESULTS
ZERO_RESULTS
ZERO_RESULTS
ZERO_RESULTS
ZERO_RESULTS
ZERO_RESULTS
ZERO_RESULTS


In [15]:
count = 0
while count < limit:
    location = df_uncod.iloc[count, df_uncod.columns.get_loc('Location')]
#     try:
#         if location != 'zz_OVER_QUERY_LIMIT' and location != 'zz_OK' and location[0:3] == 'zz_':
#             print ('Reached bottleneck')
#             break
#     except:
#         pass
    idnr = df_uncod.iloc[count, df_uncod.columns.get_loc('DataId')]
    address = df_uncod.iloc[count, df_uncod.columns.get_loc('Address')]
    city = df_uncod.iloc[count, df_uncod.columns.get_loc('City')]
    state = df_uncod.iloc[count, df_uncod.columns.get_loc('State')]
    zipcode = int(df_uncod.iloc[count, df_uncod.columns.get_loc('PostalCode')])

    # construct query to api
    if city not in city_exceptions and state not in state_exceptions:
        city = city.replace(' Area', '')
        query = ', '.join((address, city, state))
    elif city not in city_exceptions and state in state_exceptions:
        city = city.replace(' Area', '')
        query = ', '.join((address, city))
    elif city in city_exceptions and state not in state_exceptions:
        query = ', '.join((address, state))
    else:
        query = address

    # Request to googleapis
    print ('\nQuery', query)
    url = serviceurl + urllib.parse.urlencode({"address": query, "key": API_key})
    uh = urllib.request.urlopen(url, context=scontext)
    entry = uh.read()
    count += 1
    print ('Retrieved',len(entry))
    try:
        js = json.loads(entry)
        # print (js) #  Uncomment this line to print js in case unicode causes an error
    except:
        continue

    # Read status
    if 'status' not in js:
        print ('==== Failure To Retrieve (no status) ====\n')
        continue
    if js['status'] != 'OK':
        df.loc[df.DataId == idnr, 'Location'] = 'zz_' + js['status']
        print ('==== Failure To Retrieve (' + js['status'] + ') ====\n')
        continue

    # js['results'] == 'OK', read data
    lat = js['results'][0]['geometry']['location']['lat']
    lng = js['results'][0]['geometry']['location']['lng']
    location = 'POINT(' + str(lat) + ' ' + str(lng) + ')'
    where = js['results'][0]['formatted_address']
    address_components = js['results'][0]['address_components']
    print("Details:", lat, lng, location, where)
    try:
        component_postal = list(filter(lambda x: x['types'] == [ 'postal_code' ], address_components))
        postal = int(component_postal[0]['long_name'])
        print ('Postal code found in js.')
        if postal != zipcode:
            location = location + '_PostalCode_' + str(zipcode)
            #print ('Postal codes do not match: ', postal, zipcode, where, '\n')
        else:
            print ('Postal code matched.')
    except:
        print ('Postal code not found in js.')
        pass
    # Write results to dataframe
    df.loc[df.DataId == idnr, 'lat'] = lat
    df.loc[df.DataId == idnr, 'lng'] = lng
    df.loc[df.DataId == idnr, 'Location'] = location
    #print ('Retrieved #', count, ': ', where, '\n')

#     if count%500 == 0:
#         pickle.dump(df, open(FILE, 'wb'))
#         print ('Pickled #', count)
    time.sleep(0.03)


Query 1188 GALESMORE Court, Westlake Village, CA
Retrieved 52
==== Failure To Retrieve (ZERO_RESULTS) ====


Query 1611 Jennings (3rd) Street, Madera, CA
Retrieved 52
==== Failure To Retrieve (ZERO_RESULTS) ====


Query 21274 Ridgedale Drive, Lake Mathews, CA
Retrieved 52
==== Failure To Retrieve (ZERO_RESULTS) ====


Query 363 Cedar / Blue Jay Canyon Court, Blue Jay, CA
Retrieved 52
==== Failure To Retrieve (ZERO_RESULTS) ====


Query 17465 Vanderhill Circle, Lake Mathews, CA
Retrieved 52
==== Failure To Retrieve (ZERO_RESULTS) ====


Query 718 Burnt Ranch (Lot 419) Way, Chico, CA
Retrieved 52
==== Failure To Retrieve (ZERO_RESULTS) ====


Query 10574 SUNLAND BLVD #5 Boulevard, Sunland, CA
Retrieved 52
==== Failure To Retrieve (ZERO_RESULTS) ====


Query 13140 Del Monte  M16-52G Drive, Seal Beach, CA
Retrieved 52
==== Failure To Retrieve (ZERO_RESULTS) ====


Query 8074 Tulsa Place, Rancho Cucamonga, CA
Retrieved 52
==== Failure To Retrieve (ZERO_RESULTS) ====



In [20]:
(df.lat == 0).sum()
# After coding, there are still 11 entries with no geoinfo returned.

9

In [17]:
# Save the processed df back to pickle. 
pickle.dump(df, open('Data_Step1_8', 'wb'))

In [19]:
df.shape

(22112, 103)